### 3.3 Hypothesis Testing

Repetition of Task 3.2 using a different model for each dataset. We will use hypothesis testing to compare the performance of the models developed in this task and Task 3.2.

#### Markdown Question: 
**Question**: What was the hypothesis you rejected for both datasets in this task? How does p-value affect your decision?